## Linear Regression Demonstration

Using `regression-inference` package

In [1]:
from regression_inference import LinearRegression, summary

In [2]:
import numpy as np
import pandas as pd

In [3]:
#© Copyright 2007 - 2025, scikit-learn developers (BSD License).
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame = True).frame

### Model Fitting

- Fit the Linear Regression on the training set

In [4]:
data['const'] = np.ones(len(data))

features = data[[
    'const', 'MedInc', 'HouseAge', 'AveRooms',
    'AveBedrms', 'AveOccup', 'MedHouseVal'
]].dropna()

X = features.drop(columns=['MedHouseVal'])
y = features['MedHouseVal']

lm = LinearRegression().fit(X=X, y=y, cov_type=None, alpha=0.05)

In [5]:
# Printing the fitted model calls summary(model)

print(lm)
#print(summary(lm))

OLS Regression Results
---------------------------------------------
Dependent:                        MedHouseVal
---------------------------------------------
 
const                              -0.3857***
                                     (0.0247)
 
MedInc                              0.5374***
                                     (0.0041)
 
HouseAge                            0.0159***
                                     (0.0004)
 
AveRooms                           -0.2139***
                                     (0.0060)
 
AveBedrms                           0.9985***
                                     (0.0295)
 
AveOccup                           -0.0047***
                                     (0.0005)

---------------------------------------------
R-squared                               0.539
Adjusted R-squared                      0.539
F Statistic                          4830.447
Observations                        20640.000
Log Likelihood                     -24244.36

### Model Predictions


In [7]:
lm.feature_names[1:]

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'AveOccup'], dtype='object')

In [8]:
# All predictions are in order of model.feature_names[1:]

lm.predict( X = [8.3252, 41, 6.98412698,  1.02380952, 2.55555556] )

np.float64(4.255838465151851)

In [9]:
# Predict new values with inference
# return_table requires 2d array

prediction = lm.predict(X = [[8.3252, 41, 6.98412698,  1.02380952, 2.55555556]], return_table = True )

pd.DataFrame(prediction)

,features,prediction,std_error,t_statistic,P>|t|,ci_low_0.05,ci_high_0.05
0,"{'MedInc': '8.33', 'HouseAge': '41.00', 'AveRo...",4.2558,0.0156,272.8757,0.000,4.2253,4.2864


In [10]:
prediction_set = [
    [[8.3252, 41, 6.98412698, 1.02380952, 2.55555556]],
    [[8.3014, 21, 6.23813708, 0.97188049, 2.10984183]],
] 

predictions = pd.concat(
    [pd.DataFrame(lm.predict(X = pred, return_table=True)) for pred in prediction_set]
)

predictions

,features,prediction,std_error,t_statistic,P>|t|,ci_low_0.05,ci_high_0.05
0,"{'MedInc': '8.33', 'HouseAge': '41.00', 'AveRo...",4.2558,0.0156,272.8757,0.000,4.2253,4.2864
0,"{'MedInc': '8.30', 'HouseAge': '21.00', 'AveRo...",4.0354,0.0149,270.6509,0.000,4.0062,4.0646


In [11]:
# Predict at the sample mean
  
sample_mean = (
    [X[i].mean() for i in list(lm.feature_names[1:])] # Preserves ordering
) 

prediction_set = [[sample_mean]] 

predictions = pd.concat(
    [pd.DataFrame(lm.predict(X = pred, return_table=True)) for pred in prediction_set]
)

predictions

,features,prediction,std_error,t_statistic,P>|t|,ci_low_0.05,ci_high_0.05
0,"{'MedInc': '3.87', 'HouseAge': '28.64', 'AveRo...",2.0686,0.0055,379.3685,0.000,2.0579,2.0792


In [12]:
'''
Predict each year of HouseAge holding all else at the sample mean

Maintain order of lm.feature_names[1:], ie, ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'AveOccup']
'''

prev_names, post_names = ['MedInc'], ['AveRooms', 'AveBedrms', 'AveOccup']

mean_prev, mean_post = [X[i].mean() for i in prev_names], [X[i].mean() for i in post_names]


prediction_range = np.linspace(
    X['HouseAge'].min(),
    X['HouseAge'].max(),
    1 + int(X['HouseAge'].max()) - int(X['HouseAge'].min())
)

prediction_set = [
    [mean_prev + [i] + mean_post]
    for i in prediction_range  
] 

predictions = pd.concat(
    [pd.DataFrame(lm.predict(X = pred, return_table=True)) for pred in prediction_set]
)

predictions.head()

,features,prediction,std_error,t_statistic,P>|t|,ci_low_0.05,ci_high_0.05
0,"{'MedInc': '3.87', 'HouseAge': '1.00', 'AveRoo...",1.6299,0.0133,122.1344,0.000,1.6037,1.6560
0,"{'MedInc': '3.87', 'HouseAge': '2.00', 'AveRoo...",1.6457,0.0129,127.1440,0.000,1.6204,1.6711
0,"{'MedInc': '3.87', 'HouseAge': '3.00', 'AveRoo...",1.6616,0.0125,132.4456,0.000,1.6370,1.6862
0,"{'MedInc': '3.87', 'HouseAge': '4.00', 'AveRoo...",1.6775,0.0122,138.0617,0.000,1.6537,1.7013
0,"{'MedInc': '3.87', 'HouseAge': '5.00', 'AveRoo...",1.6934,0.0118,144.0160,0.000,1.6703,1.7164


### Coefficient Inference Table

- Comprehensive regression inference 

In [13]:
pd.DataFrame(lm.inference_table())

,feature,coefficient,std_error,t_statistic,P>|t|,ci_low_0.05,ci_high_0.05
0,const,-0.3857,0.0247,-15.6176,0.000,-0.4341,-0.3373
1,MedInc,0.5374,0.0041,130.3086,0.000,0.5293,0.5455
2,HouseAge,0.0159,0.0004,36.0159,0.000,0.015,0.0167
3,AveRooms,-0.2139,0.006,-35.6918,0.000,-0.2256,-0.2021
4,AveBedrms,0.9985,0.0295,33.8376,0.000,0.9406,1.0563
5,AveOccup,-0.0047,0.0005,-8.9512,0.000,-0.0057,-0.0037


### Variance Inflation Factor

- Generate a VIF table on the models features

In [14]:
pd.DataFrame(lm.variance_inflation_factor())

,feature,VIF
0,MedInc,2.0649
1,HouseAge,1.0346
2,AveRooms,7.3916
3,AveBedrms,6.5768
4,AveOccup,1.0009


### Robust Covariance (Preview)

- Preview the effect of robust covariances without applying to the model

In [15]:
pd.DataFrame(lm.robust_se(type="HC0"))

,feature,robust_se,robust_t,robust_p,ci_low_0.05,ci_high_0.05
0,const,0.090548,-4.259892,0.000021,-0.563204,-0.208243
1,MedInc,0.010606,50.673000,0.000000,0.516644,0.558221
2,HouseAge,0.000560,28.356192,0.000000,0.014774,0.016969
3,AveRooms,0.018929,-11.297794,0.000000,-0.250961,-0.176755
4,AveBedrms,0.102282,9.761777,0.000000,0.797972,1.198933
5,AveOccup,0.001116,-4.212912,0.000025,-0.006889,-0.002514


In [16]:
pd.DataFrame(lm.robust_se(type="HC1"))

,feature,robust_se,robust_t,robust_p,ci_low_0.05,ci_high_0.05
0,const,0.090561,-4.259273,0.000021,-0.563229,-0.208217
1,MedInc,0.010607,50.665634,0.000000,0.516641,0.558224
2,HouseAge,0.000560,28.352071,0.000000,0.014774,0.016969
3,AveRooms,0.018932,-11.296152,0.000000,-0.250966,-0.176750
4,AveBedrms,0.102297,9.760358,0.000000,0.797943,1.198962
5,AveOccup,0.001116,-4.212300,0.000025,-0.006889,-0.002514


In [17]:
pd.DataFrame(lm.robust_se(type="HC2"))

,feature,robust_se,robust_t,robust_p,ci_low_0.05,ci_high_0.05
0,const,0.102204,-3.774034,0.000161,-0.586052,-0.185394
1,MedInc,0.011408,47.108947,0.000000,0.515071,0.559794
2,HouseAge,0.000576,27.566135,0.000000,0.014743,0.017000
3,AveRooms,0.020602,-10.380413,0.000000,-0.254240,-0.173476
4,AveBedrms,0.115616,8.635914,0.000000,0.771836,1.225070
5,AveOccup,0.001553,-3.026952,0.002473,-0.007746,-0.001657


In [18]:
pd.DataFrame(lm.robust_se(type="HC3"))

,feature,robust_se,robust_t,robust_p,ci_low_0.05,ci_high_0.05
0,const,0.116087,-3.322717,8.930039e-04,-0.613262,-0.158184
1,MedInc,0.012372,43.438051,0.000000e+00,0.513182,0.561683
2,HouseAge,0.000595,26.675229,0.000000e+00,0.014705,0.017038
3,AveRooms,0.022554,-9.481876,0.000000e+00,-0.258066,-0.169650
4,AveBedrms,0.131599,7.587064,3.397282e-14,0.740508,1.256398
5,AveOccup,0.002447,-1.921199,5.472040e-02,-0.009498,0.000095


### Robust Covariance (Apply on Fit)

- Apply a robust covariance to a model during fit

- Subsequent predictions will be made with the robust covariance

In [19]:
lm_robust_hc0 = LinearRegression().fit(X=X, y=y, cov_type="HC0", alpha=0.05, target_name="HouseValHC0")
lm_robust_hc1 = LinearRegression().fit(X=X, y=y, cov_type="HC1", alpha=0.05, target_name="HouseValHC1")
lm_robust_hc2 = LinearRegression().fit(X=X, y=y, cov_type="HC2", alpha=0.05, target_name="HouseValHC2")
lm_robust_hc3 = LinearRegression().fit(X=X, y=y, cov_type="HC3", alpha=0.05, target_name="HouseValHC3")

In [20]:
# Compare to the nonrobust model

print(summary(lm, lm_robust_hc0, lm_robust_hc1, lm_robust_hc2, lm_robust_hc3))

OLS Regression Results
---------------------------------------------------------------------------------------------------------
Dependent:                        MedHouseVal    HouseValHC0    HouseValHC1    HouseValHC2    HouseValHC3
---------------------------------------------------------------------------------------------------------
 
const                              -0.3857***     -0.3857***     -0.3857***     -0.3857***     -0.3857***
                                     (0.0247)       (0.0905)       (0.0906)       (0.1022)       (0.1161)
 
MedInc                              0.5374***      0.5374***      0.5374***      0.5374***      0.5374***
                                     (0.0041)       (0.0106)       (0.0106)       (0.0114)       (0.0124)
 
HouseAge                            0.0159***      0.0159***      0.0159***      0.0159***      0.0159***
                                     (0.0004)       (0.0006)       (0.0006)       (0.0006)       (0.0006)
 
AveRooms       